In [ ]:
# If not installed, run this cell. It is needed for access to GEE
#!pip install ee
#!pip show ee

In [ ]:
# If not installed, run this cell. It helps visualization of data
#!pip install geemap
#!pip show geemap

In [ ]:
# If not installed, run this cell. It helps conversion from DBF to DF
#!pip install dbfread
#!pip show dbfread

In [1]:
# Import libraries
import ee, geemap

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-jvg')

In [4]:
# HydroSHEDS v1 will be used. Specifically, its HydroBASINS
Map = geemap.Map()
hydrobasins = ee.FeatureCollection('WWF/HydroSHEDS/v1/Basins/hybas_3')
hybas_id = "HYBAS_ID"
Map.addLayer(hydrobasins, {}, 'HydroBasins')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [5]:
# Access the asset containing the calibration drainage areas
calib_basins = ee.FeatureCollection('projects/ee-jvg/assets/calibration_basins_gene')
Map.centerObject(calib_basins.geometry())
Map.addLayer(calib_basins, {"color": 'blue'}, 'Calibration Drainage Areas')

In [6]:
# TerraClimate layers filtered with "start_date" (inclusive) and "end_date" (exclusive) and clipped to "zoi"
bands = ["pr", "pet", "ro"]
terraclimate = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').select(bands).filterBounds(calib_basins)

In [21]:
projection = terraclimate.first().projection().getInfo()

In [7]:
# Let's assign year and month to each image of the resulting image collection
final_ic = terraclimate.map(lambda image: image.set("yr-mth", ee.Image(image).date().get("year").format().cat(ee.String("-")).cat(ee.Image(image).date().get("month").format())))

In [26]:
# Export a cloud-optimized GeoTIFF.
task = ee.batch.Export.image.toDrive(
    image=final_ic,
    description='TerraClimate_Dataset',
    crs=projection['wkt'],
    crsTransform=projection['transform'],
    region=calib_basins,
    fileFormat='GeoTIFF'
)
task.start()